# Linear Algebra

It is safe to say that linear algebra is the cornerstone of modern mathematical physics, providing the essential tools for understanding and solving a wide range of problems. From the study of vector spaces and transformations to eigenvalue problems and matrix operations, linear algebra forms the foundation for many physical theories. Whether in classical mechanics, quantum physics, relativity, or electrodynamics, linear algebra enables the representation of physical quantities, the formulation of physical laws, and the manipulation of complex systems in an efficient and structured way. Its concepts permeate the formulation of everything from equations of motion to the quantum states of particles, making it indispensable for both theoretical insights and practical computations.

Opposed to the widespread disbelief, vectors are not merely 1D arrays, and tensors are not simply multi-dimensional arrays either. While it’s common to represent vectors and tensors in this form for computational convenience, their true nature is far more profound. Vectors exist in an abstract vector space and only take on specific numerical values when projected onto a basis. Similarly, tensors are generalized multi-linear mappings that relate vectors, scalars, and other tensors in ways that transcend their array-like representations. Their intrinsic properties remain invariant under transformations, making them essential tools in describing physical phenomena, from forces and fields to stress and strain in materials, in a way that doesn’t depend on arbitrary choices of coordinates.

**The linear algebra submodule in `sigmaepsilon.math` provides implementations of Vectors and Tensors as they are fundamentally understood in physics.**

## Arrays

The library provides some classes for arrays, namely [Array](..\api\api_linalg.rst#Array), [JaggedArray](..\api\api_linalg.rst#JaggedArray) and [csr_matrix](..\api\api_linalg.rst#csr_matrix).

### `Array`

The `Array` class is basically the same as the `ndarray` class in NumPy, with the minor difference that it has a safe metaclass, meaning that there is a safety mechanism that prevents you from unintentionally crashing the internal behaviour of the class upon subclassing. This practically means, that you will see an error if you try to shadow a definition in any of the base classes of the class. For this reason it is safer to subclass this class rather than to directly subclass NumPy's ndarray class.

### `JaggedArray`

Jagged arrays are like matrices (2d arrays) that have rows of different length. The class wraps either a NumPy array or an Awkward array in the background, depending on the input and generalizes some matrix operations for the jagged case.

In [55]:
from sigmaepsilon.math.linalg import JaggedArray
import numpy as np

data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
arr = JaggedArray(data, cuts=[3, 3, 3])
arr

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [56]:
type(arr._wrapped)

numpy.ndarray

In [57]:
arr.is_jagged()

False

In [58]:
arr.widths()

array([3, 3, 3], dtype=int64)

The underlying array in this case is a NumPy array and it should behave like one, you can use it as a drop-in replacement. However, the constructor of the class accepts a `force_numpy` parameter, which is `True` by default. It means that the constructor will try to use NumPy whenever possible and only uses Awkward if the input is truly jagged.

In [59]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
arr = JaggedArray(data, cuts=[3, 3, 4])
arr

<Array [[1, 2, 3], [4, 5, 6], [7, 8, 9, 10]] type='3 * var * int32'>

In [60]:
type(arr._wrapped)

awkward.highlevel.Array

In [61]:
arr.is_jagged()

True

In [62]:
arr.widths()

array([3, 3, 4], dtype=int64)

The class generalizes some NumPy functions to jagged arrays, like `unique` and `concatenate`.

In [63]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 9])
arr = JaggedArray(data, cuts=[3, 3, 4])
np.unique(arr)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [64]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
arr1 = JaggedArray(data, cuts=[3, 3, 4])

data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
arr2 = JaggedArray(data, cuts=[3, 2, 6])

np.concatenate([arr1, arr2])

<Array [[1, 2, 3], [4, ...], ..., [6, 7, 8, 9, 10, 11]] type='6 * var * int32'>

For other operations, use the `to_array` method of the instance, which will return either a NumPy or an Awkward array, both of which has support for all the operations you might need, and you can use the result to initialize another `JaggedArray` instance if necessary.

In [65]:
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 9])
arr = JaggedArray(data, cuts=[3, 3, 4])
arr = JaggedArray(arr.to_array()*3)
arr

<Array [[3, 6, 9], [12, ..., 18], [21, 24, 27, 27]] type='3 * var * int32'>

You can also use the `JaggedArray` class to transform between different data formats. For the details of this transformation functions, consult the API Reference.

In [66]:
arr.to_array()  # returns a NumPy or an Awkward array
arr.to_ak()     # returns an Awkward array
arr.to_numpy()  # returns a dense NumPy array
arr.to_list()   # returns a list of lists
arr.to_csr()    # returns an instance of sigmaepsilon.math.linalg.sparse.csr_matrix
arr.to_scipy()  # returns an instance of scipy.sparse.csr_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (3, 4)>

### `csr_matrix`

The `csr_matrix` class is basically a remake of the similar class in SciPy, with the addition that it is Numba-jittable. To create an instance, you can start from a `JaggedArray` instance, or use any other input you would use with a SciPy csr matrix.

In [67]:
from scipy.sparse import csr_matrix as csr_scipy
from sigmaepsilon.math.linalg import csr_matrix

# create a JaggedArray and convert it to a scipy.sparse.csr_matrix
data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
scipy_matrix = JaggedArray(data, cuts=[3, 3, 4]).to_csr()

# create a csr_matrix instance from a dense NumPy array
scipy_matrix = csr_scipy((3, 4), dtype=np.int8).toarray()
csr_matrix(scipy_matrix)

3x4 CSR matrix of 12 values.

Instances of the `csr_matrix` class are Numba-jittable, with some attributed and methods accessible inside Numba-jitted functions, even in nopython-mode.

In [68]:
from numba import jit


@jit(nopython=True, nogil=True)
def numba_nopython(csr: csr_matrix, i: int):
    csr.data
    csr.indices
    csr.indptr
    csr.shape
    return csr.row(i), csr.irow(i)


row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
matrix = csr_scipy((data, (row, col)), shape=(3, 3))

csr = csr_matrix(matrix)
numba_nopython(csr, 0)

(array([1., 2.]), array([0, 2]))

## Reference Frames

Whether you are working with vectors or tensors, in order to represent them as one or multi-dimensional arrays, you need a reference frame. The `sigmaepsilon.math.linalg` provides you with the following classes to represent reference frames:

- [ReferenceFrame](..\api\api_linalg.rst#ReferenceFrame): The most general reference frame, allowing for arbitrary settings. Base vectors do not need to be orthogonal, nor normalized.
- [RectangularFrame](..\api\api_linalg.rst#RectangularFrame): For the case when base vectors are not normalized, but orthogonal.
- [CartesianFrame](..\api\api_linalg.rst#RectangularFrame): For frames with orthonormal base vectors.

Of course, you can just use the most general `ReferenceFrame` class and accept that some calculations might involve computing the Gram matrix, which in some cases would simplify to the identity matrix, meaning that the class is a bit overpower for your needs. Or, you can go with the `RectangularFrame` and `CartesianFrame` classes that might be a better choice for your problem and they are computationally more efficient. The good thing is that you don't have to worry about crossing the boundaries between these types, the library takes care of everything for you.

In [69]:
from sigmaepsilon.math.linalg import ReferenceFrame, RectangularFrame, CartesianFrame

Let's say you get your hands on a cartesian frame (one with orthonormal basis vectors).

In [70]:
frame = CartesianFrame(dim=3)

Now there are some operations you can apply on a frame. Some of these operations would result in another frame, which is not cartesian. Some operations would modify the frame inplace and result in the instance losing the property of orthogonality or normality. For instance, if all basis vectors were multiplied by a scalar, the new basis vectors would be still orthogonal, but not normal.

In [71]:
type(frame*2)

sigmaepsilon.math.linalg.frame.RectangularFrame

As you can see, in this case the instance created by the operation is the next most general one, which is a `RectangularFrame`. If let's say no you wanted to increment all coordinates of the basis vectors by the same scalar, there would be no guarantee of the basis vectors of the resulting frame to being either orthogonal nor normal, hence the frame created by the operation is the most general one, an instance of `ReferenceFrame`.

In [72]:
type(frame+2)

sigmaepsilon.math.linalg.frame.ReferenceFrame

 The good new is that you don't really have to worry about these things as the library will choose the most appropriate class for the output, depending on the nature of the operation.

In [73]:
type(frame)

sigmaepsilon.math.linalg.frame.CartesianFrame

In [74]:
frame *=2
type(frame)

sigmaepsilon.math.linalg.frame.RectangularFrame

In [75]:
frame +=2
type(frame)

sigmaepsilon.math.linalg.frame.ReferenceFrame

In [76]:
frame.axes

Array([[4., 2., 2.],
       [2., 4., 2.],
       [2., 2., 4.]])

You can see that the basis vectors of the frame are no longer orthogonal or normed, but they are still independent.

In [77]:
frame.is_cartesian

False

In [78]:
frame.is_rectangular

False

In [79]:
frame.is_independent

True

It is also possible to get the dual frame (aka. reciprocal frame) of a frame. This is one example where the `CartesianFrame` is computationally more effective, as the dual frame of an orthonormal frame is itself.

In [80]:
frame.dual()

Array([[ 0.375, -0.125, -0.125],
       [-0.125,  0.375, -0.125],
       [-0.125, -0.125,  0.375]])

## Vectors

As a basic example, consider two orthonormal frames in Euclidean space. The first one is the standard frame defined with the unit matrix $\mathbf{I}$, the second is obtained by rotating $\mathbf{I}$ around the $z$ axis with $90$ degrees. A vector is defined in $\mathbf{I}$ (the source) and we want to know it's components in the rotated frame (the target).

We begin by creating the coordinates of the vector as an 1d array.

In [81]:
import numpy as np

# the coordinates of a vector in the source frame
arr_source = np.array([3 ** 0.5 / 2, 0.5, 0])
arr_source

array([0.8660254, 0.5      , 0.       ])

In sigmaepsilon.math, the [ReferenceFrame](..\api\api_linalg.rst#ReferenceFrame) class provides the machinery to establish a connection between two frames. First, we create the frames as:

In [82]:
from sigmaepsilon.math.linalg import ReferenceFrame

source_frame = ReferenceFrame(dim=3)  # this is a 3 by 3 identity matrix
target_frame = source_frame.orient_new('Body', [0, 0, 90*np.pi/180],  'XYZ')

To transform the vector into the target frame, we can use the Vector object directly, and it handles everything in the background.

In [83]:
from sigmaepsilon.math.linalg import Vector

arr_target = Vector(arr_source, frame=source_frame).show(target_frame)
arr_target

array([ 0.5      , -0.8660254,  0.       ])

## Tensors

The library also provides tools to deal with tensors. The implementation has only three limitations:

* The reference frames must be linear.
* All the indices of a tensor must be either covariant or contravariant.
* The sum total of indices used in any kind of operation must be less than or equal to 26. This is because the implementation relies on the `einsum` function in NumPy, and the english alphapet has 26 characters.

Create a Tensor of order 6 in a frame with random components:

In [97]:
from sigmaepsilon.math.linalg import Tensor, ReferenceFrame
from numpy.random import rand

frame = ReferenceFrame(dim=3)
array = rand(3, 3, 3, 3, 3, 3)
A = Tensor(array, frame=frame)

Get the tensor in the dual frame:

In [98]:
A_dual = A.dual()

Create an other tensor, in this case a 5th-order one, and calculate their generalized dot product, which is a 9th-order tensor:

In [99]:
from sigmaepsilon.math.linalg import dot

array = rand(3, 3, 3, 3, 3)
B = Tensor(array, frame=frame)
C = dot(A, B, axes=[0, 0])
assert C.rank == (A.rank + B.rank - 2)

The library also provides dedicated tensor classes for the most common quantities of engineering practice.

In [100]:
from sigmaepsilon.math.linalg import Tensor2, Tensor4, Tensor2x3, Tensor4x3

## Objectivity of Tensorial Quantities

A remarkable feature of the library is that tensorial quantities (like vectors and tensors) keep their objectivity with respect to changes of their host reference frame. To show this, create a new vector.

In [101]:
arr = np.array([1, 0, 0])
frame = ReferenceFrame(dim=3)
vector = Vector(arr, frame=frame)

Now transform the hosting frame of the vector inplace.

In [102]:
frame.orient('Body', [0, 0, -90*np.pi/180],  'XYZ')
frame.axes

Array([[ 6.123234e-17, -1.000000e+00,  0.000000e+00],
       [ 1.000000e+00,  6.123234e-17,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00]])

If you now check the coordinates of the vector, you will see that they have changed, due to the changes in the hosting frame.

In [103]:
vector.array

Array([6.123234e-17, 1.000000e+00, 0.000000e+00])

However, the vector stayed the same. You can check this, by asking for the coordinates of the vector in the ambient frame (which in this case is the frame it was defined in).

In [104]:
vector.show()

array([1., 0., 0.])

The point is that if a quantity is tensorial, it doesn't matter which frame you define the quantity in, since its meaning is independent from the hosting frame. Therefore, if the hosting frame changes, the coordinates of the tensorial quantity must change as well. A more trivial example is the following:

In [105]:
arr = np.array([1, 0, 0])
frame = ReferenceFrame(dim=3)
vector = Vector(arr, frame=frame)
frame *= 2
vector.array

Array([0.5, 0. , 0. ])

It makes perfect sense. The length of the basis vectors doubled, so the coordinates shrunk to half of the original values. Resetting the basis vectors (axes) of the hosting frame, resets the coordinates of the vector as well.

In [106]:
frame.axes = np.eye(3)
vector.array

Array([1., 0., 0.])

In fact, thanks to the flexibility of the `ReferenceFrame` class, you can assign any 3x3 positive definite matrix to the axes of the frame, and the meaning of the vector wouldn't change a thing. You can check this by asking for the coordinates of the vector in a fixed frame, which can be the ambient frame as well.

In [114]:
from sigmaepsilon.math.linalg import random_posdef_matrix

random_axes = random_posdef_matrix(3) * 100
frame.axes = random_axes
vector.show()

array([1.00000000e+00, 1.68551824e-12, 4.61957139e-13])